In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import pathlib

from datetime import datetime, timedelta

import intake
import healpy
import tobac

In [2]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")

In [73]:
cat.ICON

ICON:
  args:
    path: https://data.nextgems-h2020.eu/ICON/main.yaml
  description: ICON model output
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    catalog_dir: https://data.nextgems-h2020.eu


In [44]:
cat.ICON.ngc4008.describe()

{'name': 'ngc4008',
 'container': 'xarray',
 'plugin': ['zarr'],
 'driver': ['zarr'],
 'description': '',
 'direct_access': 'forbid',
 'user_parameters': [{'name': 'time',
   'description': 'time resolution of the dataset',
   'type': 'str',
   'allowed': ['PT15M', 'PT3H', 'P1D'],
   'default': 'P1D'},
  {'name': 'zoom',
   'description': 'zoom resolution of the dataset',
   'type': 'int',
   'allowed': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
   'default': 0}],
 'metadata': {'project': 'nextGEMS',
  'experiment_id': 'nextgems_prefinal',
  'source_id': 'ICON-ESM',
  'simulation_id': 'ngc4008'},
 'args': {'chunks': None,
  'consolidated': True,
  'urlpath': '/work/bm1235/k202181/ngc4008/ngc4008_{{ time }}_{{ zoom }}.zarr'}}

In [57]:
dataset = cat.ICON.ngc4008(time="PT15M", zoom=9, chunks="auto").to_dask()

/home/b/b382728/miniconda3/envs/tobac/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [58]:
dataset

<xarray.Dataset> Size: 45TB
Dimensions:  (crs: 1, time: 596160, cell: 3145728)
Coordinates:
  * crs      (crs) float32 4B nan
  * time     (time) datetime64[ns] 5MB 2020-01-01T00:15:00 ... 2037-01-01
Dimensions without coordinates: cell
Data variables:
    pr       (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>
    qv2m     (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>
    rlut     (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>
    rsds     (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>
    sfcwind  (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>
    tas      (time, cell) float32 8TB dask.array<chunksize=(32, 524288), meta=np.ndarray>

In [60]:
dataset.rlut

<xarray.DataArray 'rlut' (time: 596160, cell: 3145728)> Size: 8TB
dask.array<open_dataset-rlut, shape=(596160, 3145728), dtype=float32, chunksize=(32, 524288), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 5MB 2020-01-01T00:15:00 ... 2037-01-01
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      toa outgoing longwave radiation
    standard_name:  toa_outgoing_longwave_flux
    units:          W m-2
    vgrid:          toa

In [61]:
lon = xr.DataArray(np.arange(0.05, 360, 0.1), dims=("lon",), name="lon", attrs=dict(units="degrees", standard_name="longitude"))
lat = xr.DataArray(np.arange(59.95, -60, -0.1), dims=("lat",), name="lat", attrs=dict(units="degrees", standard_name="latitude"))

pix = xr.DataArray(
    healpy.ang2pix(dataset.crs.healpix_nside, *np.meshgrid(lon, lat), nest=True, lonlat=True),
    coords=(lat, lon),
)


In [62]:
dataset.crs.healpix_nside

512

In [63]:
start_date = datetime(2021,1,1)
end_date = start_date + timedelta(hours=6)

In [64]:
def get_tb(olr):
    """
    This function converts outgoing longwave radiation to brightness temperatures.

    Args:
        olr(xr.DataArray or numpy array): 2D field of model output with OLR

    Returns:
        tb(xr.DataArray or numpy array): 2D field with estimated brightness temperatures
    """
    # constants
    aa = 1.228
    bb = -1.106e-3  # K−1
    # Planck constant
    sigma = 5.670374419e-8  # W⋅m−2⋅K−4

    # flux equivalent brightness temperature
    Tf = (abs(olr) / sigma) ** (1.0 / 4)
    tb = (((aa**2 + 4 * bb * Tf) ** (1.0 / 2)) - aa) / (2 * bb)
    return tb

In [65]:
dataset.rlut

<xarray.DataArray 'rlut' (time: 596160, cell: 3145728)> Size: 8TB
dask.array<open_dataset-rlut, shape=(596160, 3145728), dtype=float32, chunksize=(32, 524288), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 5MB 2020-01-01T00:15:00 ... 2037-01-01
Dimensions without coordinates: cell
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      toa outgoing longwave radiation
    standard_name:  toa_outgoing_longwave_flux
    units:          W m-2
    vgrid:          toa

In [66]:
dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).values.strides

(12582912, 4)

In [68]:
dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).compute().isel(cell=pix)

<xarray.DataArray 'rlut' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
array([[[219.84317, 219.84317, 219.84317, ..., 236.40135, 236.40135,
         236.40135],
        [237.19234, 237.19234, 237.19234, ..., 239.34753, 239.34753,
         239.34753],
        [236.68756, 242.31566, 242.31566, ..., 242.96004, 242.96004,
         237.46765],
        ...,
        [223.90195, 224.29219, 224.29219, ..., 223.95253, 223.95253,
         223.57906],
        [224.35904, 224.35904, 224.35904, ..., 223.82684, 223.82684,
         223.82684],
        [223.99907, 223.99907, 223.99907, ..., 223.79247, 223.79247,
         223.79247]],

       [[244.9099 , 244.9099 , 244.9099 , ..., 245.75055, 245.75055,
         245.75055],
        [243.62921, 243.62921, 243.62921, ..., 229.89635, 229.89635,
         229.89635],
        [240.16507, 241.03458, 241.03458, ..., 226.02873, 226.02873,
         211.26595],
...
        [203.04736, 203.81917, 203.81917, ..., 203.87927, 203.87927,
         203.77925],
        [207.79425, 207.79425, 207.79425, ..., 205.43114, 205.43114,
         205.43114],
        [206.28625, 206.28625, 206.28625, ..., 206.91765, 206.91765,
         206.91765]],

       [[147.65303, 147.65303, 147.65303, ..., 146.60814, 146.60814,
         146.60814],
        [150.42686, 150.42686, 150.42686, ..., 154.55028, 154.55028,
         154.55028],
        [153.83304, 151.3648 , 151.3648 , ..., 156.82008, 156.82008,
         158.30614],
        ...,
        [201.79779, 198.75793, 198.75793, ..., 197.83429, 197.83429,
         202.14505],
        [200.30273, 200.30273, 200.30273, ..., 197.53474, 197.53474,
         197.53474],
        [200.60289, 200.60289, 200.60289, ..., 200.53477, 200.53477,
         200.53477]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0
Attributes:
    cell_methods:   time: mean cell: mean
    component:      atmo
    grid_mapping:   crs
    long_name:      toa outgoing longwave radiation
    standard_name:  toa_outgoing_longwave_flux
    units:          W m-2
    vgrid:          toa

In [69]:
dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).compute().isel(cell=pix).values.strides

(4, 345600, 96)

In [70]:
dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).isel(cell=pix).compute().values.strides

/home/b/b382728/miniconda3/envs/tobac/lib/python3.11/site-packages/dask/utils.py:460: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  return self.fn(key)


(17280000, 14400, 4)

In [55]:
dataset.rlut.isel(time=slice(0, 24)).isel(cell=pix).values.strides

(17280000, 14400, 4)

In [8]:
bt = get_tb(dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).isel(cell=pix))

In [10]:
bt

<xarray.DataArray 'rlut' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
array([[[267.78726, 267.78726, 267.78726, ..., 275.0712 , 275.0712 ,
         275.0712 ],
        [275.41406, 275.41406, 275.41406, ..., 276.34604, 276.34604,
         276.34604],
        [275.1953 , 277.6244 , 277.6244 , ..., 277.90115, 277.90115,
         275.53333],
        ...,
        [269.59238, 269.76526, 269.76526, ..., 269.61478, 269.61478,
         269.44925],
        [269.79483, 269.79483, 269.79483, ..., 269.55905, 269.55905,
         269.55905],
        [269.6354 , 269.6354 , 269.6354 , ..., 269.54382, 269.54382,
         269.54382]],

       [[278.73685, 278.73685, 278.73685, ..., 279.09647, 279.09647,
         279.09647],
        [278.18823, 278.18823, 278.18823, ..., 272.23447, 272.23447,
         272.23447],
        [276.6987 , 277.07336, 277.07336, ..., 270.53296, 270.53296,
         263.9269 ],
...
        [260.16562, 260.5216 , 260.5216 , ..., 260.5493 , 260.5493 ,
         260.5032 ],
        [262.34583, 262.34583, 262.34583, ..., 261.26315, 261.26315,
         261.26315],
        [261.65555, 261.65555, 261.65555, ..., 261.94482, 261.94482,
         261.94482]],

       [[232.74074, 232.74074, 232.74074, ..., 232.17938, 232.17938,
         232.17938],
        [234.22137, 234.22137, 234.22137, ..., 236.39737, 236.39737,
         236.39737],
        [236.02103, 234.71893, 234.71893, ..., 237.58292, 237.58292,
         238.35449],
        ...,
        [259.58807, 258.17654, 258.17654, ..., 257.7458 , 257.7458 ,
         259.74872],
        [258.89502, 258.89502, 258.89502, ..., 257.60583, 257.60583,
         257.60583],
        [259.03433, 259.03433, 259.03433, ..., 259.00275, 259.00275,
         259.00275]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0

In [11]:
dt = 900  # in seconds
dxy = 11100  # in meter (for Latitude)

parameters_features = dict(
    dxy=dxy,
    threshold=[241, 233, 225],
    n_min_threshold=10,
    min_distance=2.5*dxy,
    target="minimum",
    position_threshold="center",
    PBC_flag="hdim_2",
    statistic={"feature_min_BT": np.nanmin},
)

parameters_segments = dict(
    threshold=241, target="minimum", PBC_flag="hdim_2", seed_3D_flag="box", seed_3D_size=11,
)

In [12]:
print(datetime.now(), f"Commencing feature detection", flush=True)
features = tobac.feature_detection_multithreshold(
    bt.to_iris(),
    **parameters_features,
)

2024-03-10 02:10:07.727029 Commencing feature detection


In [13]:
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature_min_BT,feature,time,timestr,latitude,longitude
0,0,3,0.875000,850.749999,16,241,238.717438,1,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862500,85.125000
1,0,5,1.687500,1364.593747,32,241,236.048599,2,2021-01-01 00:00:00,2021-01-01 00:00:00,59.781250,136.509375
2,0,6,0.800000,1389.333332,15,241,235.188721,3,2021-01-01 00:00:00,2021-01-01 00:00:00,59.870000,138.983333
3,0,12,1.270833,2722.625016,48,241,238.608307,4,2021-01-01 00:00:00,2021-01-01 00:00:00,59.822917,272.312502
4,0,13,0.880000,2837.360000,25,241,239.611176,5,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862000,283.786000
...,...,...,...,...,...,...,...,...,...,...,...,...
42711,23,5027,967.666667,1741.277778,18,225,216.7798,42712,2021-01-01 05:45:00,2021-01-01 05:45:00,-36.816667,174.177778
42712,23,5030,987.454545,1765.636364,22,225,221.064957,42713,2021-01-01 05:45:00,2021-01-01 05:45:00,-38.795455,176.613636
42713,23,5031,1024.136364,1751.772726,44,225,223.072266,42714,2021-01-01 05:45:00,2021-01-01 05:45:00,-42.463636,175.227273
42714,23,5032,1040.090909,1770.045455,22,225,222.739288,42715,2021-01-01 05:45:00,2021-01-01 05:45:00,-44.059091,177.054545


In [14]:
features["feature_min_BT"] = features["feature_min_BT"].to_numpy().astype(float)

print(datetime.now(), f"Commencing segmentation", flush=True)
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message="Warning: converting a masked element to nan.*",
)
warnings.filterwarnings(
    "ignore",
    # category=FutureWarning,
    message="FutureWarning: Calling float on a sing*",
)
segments, features = tobac.segmentation.segmentation(
    features, bt.to_iris(), dxy, **parameters_segments,
)

2024-03-10 02:11:13.290302 Commencing segmentation


In [15]:
features["time"] = xr.CFTimeIndex(features["time"].to_numpy()).to_datetimeindex()

In [16]:
features = tobac.utils.bulk_statistics.get_statistics_from_mask(
    features, segments, bt, statistic=dict(mean_BT=np.nanmean), default=np.nan
)

In [25]:
%load_ext line_profiler

In [26]:
from tobac.utils.bulk_statistics import get_statistics, get_statistics_from_mask

In [31]:
%lprun -f get_statistics -f get_statistics_from_mask.__wrapped__ tobac.utils.bulk_statistics.get_statistics_from_mask(features[features.frame==1], segments[1:2], bt[1:2], statistic=dict(mean_BT=np.nanmean), default=np.nan)


Timer unit: 1e-09 s

Total time: 1.92774 s
File: /home/b/b382728/python/tobac/tobac/utils/bulk_statistics.py
Function: get_statistics at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def get_statistics(
    26                                               features: pd.DataFrame,
    27                                               labels: np.ndarray[int],
    28                                               *fields: tuple[np.ndarray],
    29                                               statistic: dict[str, Union[Callable, tuple[Callable, dict]]] = {
    30                                                   "ncells": np.count_nonzero
    31                                               },
    32                                               index: Union[None, list[int]] = None,
    33                                               default: Union[None, float] = None,
    34                                          

In [35]:
bt

<xarray.DataArray 'rlut' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
array([[[267.78726, 267.78726, 267.78726, ..., 275.0712 , 275.0712 ,
         275.0712 ],
        [275.41406, 275.41406, 275.41406, ..., 276.34604, 276.34604,
         276.34604],
        [275.1953 , 277.6244 , 277.6244 , ..., 277.90115, 277.90115,
         275.53333],
        ...,
        [269.59238, 269.76526, 269.76526, ..., 269.61478, 269.61478,
         269.44925],
        [269.79483, 269.79483, 269.79483, ..., 269.55905, 269.55905,
         269.55905],
        [269.6354 , 269.6354 , 269.6354 , ..., 269.54382, 269.54382,
         269.54382]],

       [[278.73685, 278.73685, 278.73685, ..., 279.09647, 279.09647,
         279.09647],
        [278.18823, 278.18823, 278.18823, ..., 272.23447, 272.23447,
         272.23447],
        [276.6987 , 277.07336, 277.07336, ..., 270.53296, 270.53296,
         263.9269 ],
...
        [260.16562, 260.5216 , 260.5216 , ..., 260.5493 , 260.5493 ,
         260.5032 ],
        [262.34583, 262.34583, 262.34583, ..., 261.26315, 261.26315,
         261.26315],
        [261.65555, 261.65555, 261.65555, ..., 261.94482, 261.94482,
         261.94482]],

       [[232.74074, 232.74074, 232.74074, ..., 232.17938, 232.17938,
         232.17938],
        [234.22137, 234.22137, 234.22137, ..., 236.39737, 236.39737,
         236.39737],
        [236.02103, 234.71893, 234.71893, ..., 237.58292, 237.58292,
         238.35449],
        ...,
        [259.58807, 258.17654, 258.17654, ..., 257.7458 , 257.7458 ,
         259.74872],
        [258.89502, 258.89502, 258.89502, ..., 257.60583, 257.60583,
         257.60583],
        [259.03433, 259.03433, 259.03433, ..., 259.00275, 259.00275,
         259.00275]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0

In [36]:
%time bt[1].values.ravel()

CPU times: user 518 µs, sys: 45 µs, total: 563 µs
Wall time: 571 µs


array([278.73685, 278.73685, 278.73685, ..., 269.48782, 269.48782,
       269.48782], dtype=float32)

In [37]:
%time bt[1].mean()

CPU times: user 7.21 ms, sys: 106 µs, total: 7.31 ms
Wall time: 6.21 ms


<xarray.DataArray 'rlut' ()> Size: 4B
array(276.74435, dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2021-01-01T00:15:00

In [45]:
bt.values.strides

(17280000, 14400, 4)

In [41]:
bt

<xarray.DataArray 'rlut' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
array([[[267.78726, 267.78726, 267.78726, ..., 275.0712 , 275.0712 ,
         275.0712 ],
        [275.41406, 275.41406, 275.41406, ..., 276.34604, 276.34604,
         276.34604],
        [275.1953 , 277.6244 , 277.6244 , ..., 277.90115, 277.90115,
         275.53333],
        ...,
        [269.59238, 269.76526, 269.76526, ..., 269.61478, 269.61478,
         269.44925],
        [269.79483, 269.79483, 269.79483, ..., 269.55905, 269.55905,
         269.55905],
        [269.6354 , 269.6354 , 269.6354 , ..., 269.54382, 269.54382,
         269.54382]],

       [[278.73685, 278.73685, 278.73685, ..., 279.09647, 279.09647,
         279.09647],
        [278.18823, 278.18823, 278.18823, ..., 272.23447, 272.23447,
         272.23447],
        [276.6987 , 277.07336, 277.07336, ..., 270.53296, 270.53296,
         263.9269 ],
...
        [260.16562, 260.5216 , 260.5216 , ..., 260.5493 , 260.5493 ,
         260.5032 ],
        [262.34583, 262.34583, 262.34583, ..., 261.26315, 261.26315,
         261.26315],
        [261.65555, 261.65555, 261.65555, ..., 261.94482, 261.94482,
         261.94482]],

       [[232.74074, 232.74074, 232.74074, ..., 232.17938, 232.17938,
         232.17938],
        [234.22137, 234.22137, 234.22137, ..., 236.39737, 236.39737,
         236.39737],
        [236.02103, 234.71893, 234.71893, ..., 237.58292, 237.58292,
         238.35449],
        ...,
        [259.58807, 258.17654, 258.17654, ..., 257.7458 , 257.7458 ,
         259.74872],
        [258.89502, 258.89502, 258.89502, ..., 257.60583, 257.60583,
         257.60583],
        [259.03433, 259.03433, 259.03433, ..., 259.00275, 259.00275,
         259.00275]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0

In [20]:
segments

Segmentation Mask (1),time,latitude,longitude
Shape,24,1200,3600
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


In [23]:
xr.DataArray.from_iris(segments)

<xarray.DataArray 'segmentation_mask' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
[103680000 values with dtype=int32]
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0
Attributes:
    long_name:  segmentation_mask

In [21]:
bt

<xarray.DataArray 'rlut' (time: 24, lat: 1200, lon: 3600)> Size: 415MB
array([[[267.78726, 267.78726, 267.78726, ..., 275.0712 , 275.0712 ,
         275.0712 ],
        [275.41406, 275.41406, 275.41406, ..., 276.34604, 276.34604,
         276.34604],
        [275.1953 , 277.6244 , 277.6244 , ..., 277.90115, 277.90115,
         275.53333],
        ...,
        [269.59238, 269.76526, 269.76526, ..., 269.61478, 269.61478,
         269.44925],
        [269.79483, 269.79483, 269.79483, ..., 269.55905, 269.55905,
         269.55905],
        [269.6354 , 269.6354 , 269.6354 , ..., 269.54382, 269.54382,
         269.54382]],

       [[278.73685, 278.73685, 278.73685, ..., 279.09647, 279.09647,
         279.09647],
        [278.18823, 278.18823, 278.18823, ..., 272.23447, 272.23447,
         272.23447],
        [276.6987 , 277.07336, 277.07336, ..., 270.53296, 270.53296,
         263.9269 ],
...
        [260.16562, 260.5216 , 260.5216 , ..., 260.5493 , 260.5493 ,
         260.5032 ],
        [262.34583, 262.34583, 262.34583, ..., 261.26315, 261.26315,
         261.26315],
        [261.65555, 261.65555, 261.65555, ..., 261.94482, 261.94482,
         261.94482]],

       [[232.74074, 232.74074, 232.74074, ..., 232.17938, 232.17938,
         232.17938],
        [234.22137, 234.22137, 234.22137, ..., 236.39737, 236.39737,
         236.39737],
        [236.02103, 234.71893, 234.71893, ..., 237.58292, 237.58292,
         238.35449],
        ...,
        [259.58807, 258.17654, 258.17654, ..., 257.7458 , 257.7458 ,
         259.74872],
        [258.89502, 258.89502, 258.89502, ..., 257.60583, 257.60583,
         257.60583],
        [259.03433, 259.03433, 259.03433, ..., 259.00275, 259.00275,
         259.00275]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2021-01-01 ... 2021-01-01T05:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0

In [18]:
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature_min_BT,feature,time,timestr,latitude,longitude,ncells,mean_BT
0,0,3,0.875000,850.749999,16,241,238.717438,1,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862500,85.125000,19,239.813217
1,0,5,1.687500,1364.593747,32,241,236.048599,2,2021-01-01 00:00:00,2021-01-01 00:00:00,59.781250,136.509375,33,238.720459
2,0,6,0.800000,1389.333332,15,241,235.188721,3,2021-01-01 00:00:00,2021-01-01 00:00:00,59.870000,138.983333,25,238.400024
3,0,12,1.270833,2722.625016,48,241,238.608307,4,2021-01-01 00:00:00,2021-01-01 00:00:00,59.822917,272.312502,49,239.906113
4,0,13,0.880000,2837.360000,25,241,239.611176,5,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862000,283.786000,26,240.199493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42711,23,5027,967.666667,1741.277778,18,225,216.779800,42712,2021-01-01 05:45:00,2021-01-01 05:45:00,-36.816667,174.177778,78,231.547394
42712,23,5030,987.454545,1765.636364,22,225,221.064957,42713,2021-01-01 05:45:00,2021-01-01 05:45:00,-38.795455,176.613636,70,228.642593
42713,23,5031,1024.136364,1751.772726,44,225,223.072266,42714,2021-01-01 05:45:00,2021-01-01 05:45:00,-42.463636,175.227273,657,232.815216
42714,23,5032,1040.090909,1770.045455,22,225,222.739288,42715,2021-01-01 05:45:00,2021-01-01 05:45:00,-44.059091,177.054545,1216,232.222198


In [22]:
features.ncells.max()

14968

In [15]:
olr = dataset.rlut.sel(time=slice(start_date, end_date-timedelta(minutes=1))).isel(cell=pix)

In [16]:
features = tobac.utils.bulk_statistics.get_statistics_from_mask(
    features, segments, olr, statistic=dict(mean_OLR=np.nanmean, min_OLR=np.nanmin), default=np.nan
)

In [17]:
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature_min_BT,feature,time,timestr,latitude,longitude,ncells,mean_BT,mean_OLR,min_OLR
0,0,3,0.875000,850.749999,16,241,238.717438,1,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862500,85.125000,19,239.813217,161.138123,158.600250
1,0,5,1.687500,1364.593747,32,241,236.048599,2,2021-01-01 00:00:00,2021-01-01 00:00:00,59.781250,136.509375,33,238.720459,159.032394,153.317749
2,0,6,0.800000,1389.333332,15,241,235.188721,3,2021-01-01 00:00:00,2021-01-01 00:00:00,59.870000,138.983333,25,238.400024,158.414948,150.987167
3,0,12,1.270833,2722.625016,48,241,238.608307,4,2021-01-01 00:00:00,2021-01-01 00:00:00,59.822917,272.312502,49,239.906113,161.317993,158.693359
4,0,13,0.880000,2837.360000,25,241,239.611176,5,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862000,283.786000,26,240.199493,161.888611,160.706055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,3,4728,920.266667,2136.800000,15,225,222.936691,6957,2021-01-01 00:45:00,2021-01-01 00:45:00,-32.076667,213.730000,80,229.945343,142.646744,129.251587
6957,3,4734,968.653772,2944.986481,517,225,199.981400,6958,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.915377,294.548648,854,221.187607,127.608101,91.417076
6958,3,4735,964.043956,1692.164823,91,225,220.707275,6959,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.454396,169.266482,555,230.191772,143.110580,125.741013
6959,3,4737,1028.250000,1758.000007,32,225,223.346283,6960,2021-01-01 00:45:00,2021-01-01 00:45:00,-42.875000,175.850001,1544,232.706650,147.694870,130.136978


In [18]:
from iris.analysis.cartography import area_weights
segment_slice = segments[0]
segment_slice.coord("latitude").guess_bounds()
segment_slice.coord("longitude").guess_bounds()
area = area_weights(segment_slice, normalize=False)

/home/b/b382728/miniconda3/envs/tobac/lib/python3.11/site-packages/iris/analysis/cartography.py:429: IrisDefaultingWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn(


In [19]:
area = xr.DataArray(area, coords=dict(lat=bt.lat, lon=bt.lon), dims=["lat", "lon"])

In [20]:
bt.coords

Coordinates:
  * time     (time) datetime64[ns] 32B 2021-01-01 ... 2021-01-01T00:45:00
  * lat      (lat) float64 10kB 59.95 59.85 59.75 59.65 ... -59.75 -59.85 -59.95
  * lon      (lon) float64 29kB 0.05 0.15 0.25 0.35 ... 359.7 359.8 359.9 360.0

In [21]:
features = tobac.utils.bulk_statistics.get_statistics_from_mask(
    features, segments, area, statistic=dict(area=np.nansum), default=np.nan
)

In [22]:
precip = dataset.pr.sel(time=slice(start_date, end_date-timedelta(minutes=1))).isel(cell=pix) * 3.6e3
    
features = tobac.utils.bulk_statistics.get_statistics_from_mask(
    features, segments, precip, statistic=dict(max_precip=np.nanmax), default=np.nan
)
features = tobac.utils.bulk_statistics.get_statistics_from_mask(
    features, segments, precip * area.values, statistic=dict(total_precip=np.nansum), default=np.nan
)

In [23]:
features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature_min_BT,feature,time,timestr,latitude,longitude,ncells,mean_BT,mean_OLR,min_OLR,area,max_precip,total_precip
0,0,3,0.875000,850.749999,16,241,238.717438,1,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862500,85.125000,19,239.813217,161.138123,158.600250,1.179182e+09,0.126329,9.449939e+07
1,0,5,1.687500,1364.593747,32,241,236.048599,2,2021-01-01 00:00:00,2021-01-01 00:00:00,59.781250,136.509375,33,238.720459,159.032394,153.317749,2.051549e+09,0.003571,1.941722e+06
2,0,6,0.800000,1389.333332,15,241,235.188721,3,2021-01-01 00:00:00,2021-01-01 00:00:00,59.870000,138.983333,25,238.400024,158.414948,150.987167,1.553055e+09,0.000075,9.309526e+03
3,0,12,1.270833,2722.625016,48,241,238.608307,4,2021-01-01 00:00:00,2021-01-01 00:00:00,59.822917,272.312502,49,239.906113,161.317993,158.693359,3.042586e+09,0.360640,5.221084e+08
4,0,13,0.880000,2837.360000,25,241,239.611176,5,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862000,283.786000,26,240.199493,161.888611,160.706055,1.612480e+09,0.224961,3.390769e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,3,4728,920.266667,2136.800000,15,225,222.936691,6957,2021-01-01 00:45:00,2021-01-01 00:45:00,-32.076667,213.730000,80,229.945343,142.646744,129.251587,8.384182e+09,0.078158,4.619744e+07
6957,3,4734,968.653772,2944.986481,517,225,199.981400,6958,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.915377,294.548648,854,221.187607,127.608101,91.417076,8.414494e+10,128.677322,9.525132e+10
6958,3,4735,964.043956,1692.164823,91,225,220.707275,6959,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.454396,169.266482,555,230.191772,143.110580,125.741013,5.532816e+10,56.762577,9.993481e+10
6959,3,4737,1028.250000,1758.000007,32,225,223.346283,6960,2021-01-01 00:45:00,2021-01-01 00:45:00,-42.875000,175.850001,1544,232.706650,147.694870,130.136978,1.393920e+11,80.525650,1.166623e+11


In [24]:
xr.open_dataset(list(pathlib.Path("/scratch/b/b382728/tobac_features/2021/01/01/").glob("*20210101-00*_ICON_feature_mask_file.nc"))[0]).drop_vars("all_feature_labels").drop_dims(["time","lat","lon"]).to_dataframe()

,frame,idx,y,x,detection_pixel_count,threshold_value,min_BT,time_feature,timestr,latitude,longitude,segmentation_pixel_count,mean_BT,mean_OLR,min_OLR,area,max_precip,total_precip
feature,,,,,,,,,,,,,,,,,,
1,0,3,0.875000,850.749999,16,241,238.717438,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862500,85.125000,19,239.813217,161.138123,158.600250,1.179182e+09,0.126329,9.449939e+07
2,0,5,1.687500,1364.593747,32,241,236.048599,2021-01-01 00:00:00,2021-01-01 00:00:00,59.781250,136.509375,33,238.720459,159.032394,153.317749,2.051549e+09,0.003571,1.941722e+06
3,0,6,0.800000,1389.333332,15,241,235.188721,2021-01-01 00:00:00,2021-01-01 00:00:00,59.870000,138.983333,25,238.400024,158.414948,150.987167,1.553055e+09,0.000075,9.309526e+03
4,0,12,1.270833,2722.625016,48,241,238.608307,2021-01-01 00:00:00,2021-01-01 00:00:00,59.822917,272.312502,49,239.906113,161.317993,158.693359,3.042586e+09,0.360640,5.221084e+08
5,0,13,0.880000,2837.360000,25,241,239.611176,2021-01-01 00:00:00,2021-01-01 00:00:00,59.862000,283.786000,26,240.199493,161.888611,160.706055,1.612480e+09,0.224961,3.390769e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6957,3,4728,920.266667,2136.800000,15,225,222.936691,2021-01-01 00:45:00,2021-01-01 00:45:00,-32.076667,213.730000,80,229.945343,142.646744,129.251587,8.384182e+09,0.078158,4.619744e+07
6958,3,4734,968.653772,2944.986481,517,225,199.981400,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.915377,294.548648,854,221.187607,127.608101,91.417076,8.414494e+10,128.677322,9.525132e+10
6959,3,4735,964.043956,1692.164823,91,225,220.707275,2021-01-01 00:45:00,2021-01-01 00:45:00,-36.454396,169.266482,555,230.191772,143.110580,125.741013,5.532816e+10,56.762577,9.993481e+10


In [25]:
xr.open_dataset(list(pathlib.Path("/scratch/b/b382728/tobac_features/2031/01/01/").glob("*20310101-00*_ICON_feature_mask_file.nc"))[0]).drop_vars("all_feature_labels").drop_dims(["time","lat","lon"]).to_dataframe()

,frame,idx,y,x,detection_pixel_count,threshold_value,min_BT,time_feature,timestr,latitude,longitude,segmentation_pixel_count,mean_BT,mean_OLR,min_OLR,area,max_precip,total_precip
feature,,,,,,,,,,,,,,,,,,
1,0,1,26.223684,21.326364,380,241,232.757050,2031-01-01 00:00:00,2031-01-01 00:00:00,57.327632,2.182636,234,236.727158,155.217194,146.837982,1.503897e+10,0.045324,1.089687e+08
2,0,5,7.937700,1012.344006,626,241,236.275467,2031-01-01 00:00:00,2031-01-01 00:00:00,59.156230,101.284401,630,238.691193,158.965332,153.853287,3.990252e+10,0.199324,3.078989e+09
3,0,6,1.090909,1142.545455,11,241,240.051239,2031-01-01 00:00:00,2031-01-01 00:00:00,59.840909,114.304545,29,240.286896,162.059219,159.610229,1.815124e+09,0.076222,1.024403e+08
4,0,7,3.282609,1175.934790,46,241,237.844772,2031-01-01 00:00:00,2031-01-01 00:00:00,59.621739,117.643479,48,239.528610,160.586349,157.030853,3.001574e+09,0.171886,2.554096e+08
5,0,8,5.943750,1297.049992,160,241,236.966873,2031-01-01 00:00:00,2031-01-01 00:00:00,59.355625,129.754999,166,239.564056,160.656647,155.365402,1.044709e+10,0.159148,8.797751e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7260,3,4874,899.000000,432.000000,13,225,222.875656,2031-01-01 00:45:00,2031-01-01 00:45:00,-29.950000,43.250000,215,231.961349,146.351669,129.292984,2.307034e+10,5.786826,2.563444e+10
7261,3,4877,926.506494,1839.194808,77,225,218.622055,2031-01-01 00:45:00,2031-01-01 00:45:00,-32.700649,183.969481,508,232.303055,147.071701,122.103119,5.222694e+10,55.904854,1.685451e+11
7262,3,4884,990.182573,1592.655609,241,225,205.168915,2031-01-01 00:45:00,2031-01-01 00:45:00,-39.068257,159.315561,329,217.935318,121.868530,100.194824,3.156453e+10,63.430824,1.355624e+11
